In [1]:
# Parameters
DB_PATH = "C:\\Users\\Nihil\\AppData\\Local\\Temp\\pytest-of-Nihil\\pytest-29\\test_notebooks_execute_control0\\interactions.db"


# Control Center Playground


This notebook mirrors the Control Center widgets by querying the FastAPI `/api/control` endpoints and summarizing service health alongside interaction telemetry pulled from SQLite. Use it as a quick sanity check after launching the backend, Ops Deck, and Playground UI.

In [2]:
DB_PATH = globals().get('DB_PATH', '../data/interactions.db')
CONTROL_STATUS_URL = globals().get('CONTROL_STATUS_URL', 'http://localhost:8000/api/control/status')
OUTPUT_DIR = globals().get('OUTPUT_DIR', './_papermill')

In [3]:
from pathlib import Path
import json
import sqlite3
from statistics import mean
from urllib import request
from urllib.error import URLError
import pandas as pd

OUTPUT_DIR = Path(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
conn = sqlite3.connect(DB_PATH)
try:
    df = pd.read_sql_query("SELECT user_prompt_text, created_at FROM interactions", conn)
finally:
    conn.close()

prompt_count = len(df)
latest_prompts = df.tail(3) if prompt_count else pd.DataFrame(columns=['user_prompt_text', 'created_at'])
prompt_count, latest_prompts

(2,
     user_prompt_text            created_at
 0  Quickstart prompt  2025-01-01T00:00:00Z
 1          Follow-up  2025-01-02T00:00:00Z)

In [5]:
fallback_status = {
    'services': [
        {'name': 'backend', 'state': 'unknown', 'runtime': 'windows'},
        {'name': 'frontend', 'state': 'unknown', 'runtime': 'windows'},
        {'name': 'datalab', 'state': 'unknown', 'runtime': 'linux'}
    ],
    'processes': [],
    'network': {'hostname': 'offline', 'platform': 'unknown', 'uptime': 0, 'bytes_sent': 0, 'bytes_recv': 0, 'interfaces': {}},
    'logs': {},
    'timestamp': 0.0
}

try:
    with request.urlopen(CONTROL_STATUS_URL, timeout=2.0) as response:
        status_payload = json.loads(response.read().decode('utf-8'))
except URLError as exc:
    status_payload = dict(fallback_status)
    status_payload['fallback_reason'] = str(exc)

service_states = [f"{svc['name']} ({svc.get('runtime', 'n/a')}): {svc['state']}" for svc in status_payload.get('services', [])]
service_states

['backend (windows): unknown',
 'frontend (windows): unknown',
 'datalab (linux): unknown']

In [6]:
latencies = status_payload.get('logs', {}).get('backend', [])
sample_latency_ms = mean([len(line) for line in latencies]) if latencies else 0
summary = {
    'prompt_count': int(prompt_count),
    'service_states': service_states,
    'sample_latency_ms': round(sample_latency_ms, 2),
}
snapshot_path = OUTPUT_DIR / 'control_center_snapshot.json'
snapshot_path.write_text(json.dumps(summary, indent=2), encoding='utf-8')
summary

{'prompt_count': 2,
 'service_states': ['backend (windows): unknown',
  'frontend (windows): unknown',
  'datalab (linux): unknown'],
 'sample_latency_ms': 0}